<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit_Kleinkinderkennung/blob/main/Rechenzeitanalyse_Inferenzanwendung_Bild_mit_Einteilung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Verbindung zu Google Drive herstellen. 

Für Zugriff auf Datensatz,  zum Laden der Modelle und Speichern der untersuchten Bilder. 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installation Tensorflow Object Detection API

Da die Tensorflow Object Detection API nicht auf Colab vorinstalliert ist. Muss diese bei jeder neuen Laufzeit neu installiert werden.
Dies geschieht in meheren Schritten. 

Zuerst wird Tensorflow installiert. Aufgrund verschiedener Installationsprobleme hat sich die Version 2.6.0 für mich als zuverlässigste herausgestellt. 

In [ ]:
!pip install tensorflow=="2.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564.4 MB 2.6 kB/s 
     |████████████████████████████████| 14.8 MB 7.9 MB/s 
     |████████████████████████████████| 132 kB 46.7 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=53ec6018da9af04f720eded3490b201ea5e4002d621b206b27e5bbff4a38490a
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=11582e83871ff0130036957cd7cc4d6d6a72d58fa92fb6890ab97245d822a2c6
  Stored in directory: /root/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=4e9330d529b8c138e77a1fe8c0fb3a141914d8433433f560e673ba8458c8a7d6
  Stored in directory: /root/.cache/pip/wheels/62/7

Wenn noch nicht vorhanden, dann werden (via git clone) das tensorflow/models repository von Github geklont. Einiger der Dateien daraus werden für die tatsächliche Installation benötigt (im nächsten Schritt).  

In [ ]:
import pathlib
import os
 
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3598, done.
remote: Counting objects: 100% (3598/3598), done.
remote: Compressing objects: 100% (2995/2995), done.
remote: Total 3598 (delta 949), reused 1533 (delta 549), pack-reused 0
Receiving objects: 100% (3598/3598), 47.09 MiB | 17.12 MiB/s, done.
Resolving deltas: 100% (949/949), done.


Da eine parallele Überwachung von Trainingsmetriken (loss) und Validierungs-Metriken (loss, map, usw.) nicht über colab möglich ist, evaluiere ich das Modell nach abgeschlossenem Trainingslauf. Dazu müssen zwei Anpassungen an dieser Stelle vorgenommen werden. 

* Anzahl der per default gepeicherten Checkpoints erhöhen. Statt 7 checkpoints nun bis zu 200 Checkpoints gespeichert. Anpassung in model_lib_v2, Zeile 443
* Anpassung, dass alle gespeicherten Checkpoints ausgewertet werden (und nicht nur einer), model_lib_v2 ,  Z.1135

nach: https://thachngoctran.medium.com/make-tensorflows-object-detection-validation-a-true-post-process-c45785f08d3a


Nach diesr Anpassung erfolgt die eigentliche Installation der Tensorflow-Object Detection API (davor noch setup.py Datei kopieren).

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696518 sha256=1eaf2f6858e56da7061bc7d674b90eb815880872b4bf23162cce7a1b68612a55
  Stored in directory: /tmp/pip-ephem-wheel-cache-pd6ko7pw/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=09414db5a5e86b17dd156ebdc1536a3edc4ddd495871a04837e6b82a571da612
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44009 sha256=938f1d56d85873fa485736795a726d1b14d15b1d47408842be353a003245607c
  Stored in directory: /root/.cache/pip/wheels/d6/e5/b1/6b151d9b535ee50aaa6ab27d145a0104b6df02e5636f0376da
  Created wheel for docopt: filename=docopt-

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


Nun ist die Installation der Tensorflow OD API abgeschlossen und kann mit dem *model_builder_tf2_test.py* Skript getestet werden. 

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

# Modell & Labelmap laden

In [ ]:
# LabelMap von meinem GitHub-Account laden
%cd /content/

!wget https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt

/content
--2022-11-21 22:10:14--  https://raw.githubusercontent.com/MichaelSeitz98/Seminararbeit/main/labelmap.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37 [text/plain]
Saving to: ‘labelmap.pbtxt’

labelmap.pbtxt      100%[===================>]      37  --.-KB/s    in 0s      

2022-11-21 22:10:14 (2.10 MB/s) - ‘labelmap.pbtxt’ saved [37/37]



In [ ]:
labelmap_path = '/content/labelmap.pbtxt'

saved_model_dir = '/content/drive/MyDrive/training_effdetD0_8_20000_100/inference_graph/saved_model'
speicherPfadEvalBilder = '/content/drive/MyDrive/training_effdetD0_8_20000_100/rechenzeiten_val_NOGPU'

OUTPUT_PATH_DANGER = f'{speicherPfadEvalBilder}/Gefahr'
OUTPUT_PATH_NO_DANGER = f'{speicherPfadEvalBilder}/Keine_Gefahr'

print (OUTPUT_PATH_DANGER)
print (OUTPUT_PATH_NO_DANGER)

if (not os.path.isdir(speicherPfadEvalBilder)): 
    %mkdir {speicherPfadEvalBilder}
    print(f'Ordner erzeugt: {speicherPfadEvalBilder}')
else:
    print(f'Ordner: {speicherPfadEvalBilder} existiert schon')


%mkdir {OUTPUT_PATH_DANGER}
%mkdir {OUTPUT_PATH_NO_DANGER}

input_bilder_pfad_rechenzeiten  ='/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/validation'

/content/drive/MyDrive/training_effdetD0_8_20000_100/rechenzeiten_val_NOGPU/Gefahr
/content/drive/MyDrive/training_effdetD0_8_20000_100/rechenzeiten_val_NOGPU/Keine_Gefahr
Ordner: /content/drive/MyDrive/training_effdetD0_8_20000_100/rechenzeiten_val_NOGPU existiert schon
mkdir: cannot create directory ‘/content/drive/MyDrive/training_effdetD0_8_20000_100/rechenzeiten_val_NOGPU/Gefahr’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/training_effdetD0_8_20000_100/rechenzeiten_val_NOGPU/Keine_Gefahr’: File exists


In [ ]:
# Alle benötigte packages für die Inferenz importiert

import time
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
# Methode um aus Image ein Numpy_array zu machen 

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# Label map Category Index erzeugen
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

print(f'Catergory Index erzeugt aus LabelMap: {labelmap_path}')

Catergory Index erzeugt aus LabelMap: /content/labelmap.pbtxt


In [ ]:
# Das Modell wird hier reingeladen. Kann bis zu einer Minute dauern. 

tf.keras.backend.clear_session()
model = tf.saved_model.load(saved_model_dir)

print(f'Trainiertes Model erfolgreich geladen von: {saved_model_dir}')

Trainiertes Model erfolgreich geladen von: /content/drive/MyDrive/training_effdetD0_8_20000_100/inference_graph/saved_model


In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
# Ab hier geht dann die Erkennung los
# Jedes Bild aus Test-Eval-Ordner wird untersucht
# und in den SpeicherPfadEvalBilder abgespeichert. 

bilderCounter = 0
danger = False
CONFIDENT_SCORE_THRESHOLD_CHILD = 0.50

size_alle_bilder = []
zeiten_für_alle_bilder_gesamt = []
zeiten_für_alle_bilder_numpy = []
zeiten_für_alle_bilder_vorverarbeitung = []
zeiten_für_alle_bilder_inference = []
zeiten_für_alle_bilder_nachbearbeitung = []
namen_aller_bilder = []

for image_path in glob.glob(f'{input_bilder_pfad_rechenzeiten}/*.jpg'):
  start_time_gesamt = time.time()
 
  # Vorbereitung 
  danger = False
  print(f'aktueller image_path:{image_path}')
  start_time_pro_bild = time.time()
  bilderCounter+=1
  image_name = os.path.basename(image_path).replace(".jpg", "")

  start_time_numpy = time.time()
  image_np = load_image_into_numpy_array(image_path)
  dauer_pro_bild_numpy = time.time() - start_time_numpy

  dauer_pro_bild_vorverarbeitung = time.time()-start_time_gesamt
  print (f'Vorverarbeitungszeit:  {dauer_pro_bild_vorverarbeitung}')
  
  # Inferenz
  start_time_inference = time.time()
  output_dict = run_inference_for_single_image(model, image_np)
  dauer_pro_bild_inference = (time.time() - start_time_inference)
  print (f'Reine Inferenzzeit: {dauer_pro_bild_inference}')
  
  # Verarbeitung
  start_time_nachbearbeitung = time.time()
  classes_child = output_dict['detection_classes']
  scores_child = output_dict['detection_scores']
  boxes_child = output_dict["detection_boxes"]

  for i in range(0, len(classes_child)):
      if classes_child[i] == 1 and scores_child[i] >= CONFIDENT_SCORE_THRESHOLD_CHILD:
              danger = True
              #print ("Kleinkind detektiert!")

  outImg = vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      min_score_thresh=CONFIDENT_SCORE_THRESHOLD_CHILD,
      max_boxes_to_draw=15,
      line_thickness=3)
  
  im = Image.fromarray(outImg)
  #display(Image.fromarray(image_np))
    
  if (danger):
        im.save(f'{OUTPUT_PATH_DANGER}/{image_name}_boxed_danger.jpg')
        #print ('Hier gespeichert: ' + OUTPUT_PATH_DANGER + "/" + image_name + "_boxed.jpg")
  else:
        print('Keine Gefahr. Kein Kind entdeckt')
        im.save(f'{OUTPUT_PATH_NO_DANGER}/{image_name}_boxed_no_danger.jpg')
        #print(image_name + " gespeichert: No Danger.\n")
    
  dauer_pro_bild_nachbearbeitung = time.time()- start_time_nachbearbeitung
  print (f'Nachbearbeitungszeit: {dauer_pro_bild_nachbearbeitung}')

  dauer_pro_bild_gesamt = time.time() - start_time_gesamt
  print (f'Gesamtzeit : {dauer_pro_bild_gesamt}')
  
  zeiten_für_alle_bilder_gesamt.append(dauer_pro_bild_gesamt)
  zeiten_für_alle_bilder_numpy.append(dauer_pro_bild_numpy)
  zeiten_für_alle_bilder_vorverarbeitung.append(dauer_pro_bild_vorverarbeitung)
  zeiten_für_alle_bilder_inference.append(dauer_pro_bild_inference)
  zeiten_für_alle_bilder_nachbearbeitung.append(dauer_pro_bild_nachbearbeitung)
  size_alle_bilder.append (os.path.getsize(image_path))
  namen_aller_bilder.append (image_name)

  
#____________________________Ausgabe der Zeiten und Größe_______________________________

print("Zeiten_gesamt" + str(zeiten_für_alle_bilder_gesamt))
#print("Zeiten_numpy" + str(zeiten_für_alle_bilder_numpy))
print("Zeiten_vorverarbeitung:" + str(zeiten_für_alle_bilder_vorverarbeitung))
print("Zeiten: inference" + str(zeiten_für_alle_bilder_inference))
print("Zeiten: Nachbearbeitung: " + str(zeiten_für_alle_bilder_nachbearbeitung))
print("Größe: Größe des Bilds: " + str(size_alle_bilder))
print("Namen der verarbeiteten Bilder" + str(namen_aller_bilder))

print(f'Fertig! Modell auf {bilderCounter} Bilder angewendet und unter {speicherPfadEvalBilder} gespeichert.')

aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/validation/image1.jpg
Vorverarbeitungszeit:  0.48290371894836426
Reine Inferenzzeit: 0.04436779022216797
Nachbearbeitungszeit: 0.03188633918762207
Gesamtzeit : 0.5615131855010986
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/validation/image2.jpg
Vorverarbeitungszeit:  0.5185410976409912
Reine Inferenzzeit: 0.04960441589355469
Nachbearbeitungszeit: 0.029837369918823242
Gesamtzeit : 0.5990698337554932
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/validation/image3.jpg
Vorverarbeitungszeit:  0.44159746170043945
Reine Inferenzzeit: 0.04592633247375488
Nachbearbeitungszeit: 0.03176164627075195
Gesamtzeit : 0.5208339691162109
aktueller image_path:/content/drive/MyDrive/DS2_POOLG_80_20_20_1024px/validation/image4.jpg
Vorverarbeitungszeit:  0.4906899929046631
Reine Inferenzzeit: 0.04621458053588867
Nachbearbeitungszeit: 0.02814316749572754
Gesamtzeit : 0.566967248916626
aktuel